# Credit Modelling

## Data Exploration and Cleaning

In [112]:
import pandas as pd

In [113]:
df = pd.read_csv("loans_2007.csv", low_memory=False)

In [114]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [115]:
df.shape

(42538, 52)

In [116]:
df.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1, inplace=True)

In [117]:
df.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1, inplace=True)

In [118]:
df.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1, inplace=True)

In [119]:
df["loan_status"].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

In [120]:
a = ["Fully Paid", "Charged Off"]
df = df[df.loan_status.isin(a)]
status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}
df = df.replace(status_replace)


## Preprocessing and Feature Engineering

In [121]:
null_counts = df.isnull().sum()
null_counts[null_counts>0]

emp_length                    1036
title                           11
revol_util                      50
last_credit_pull_d               2
collections_12_mths_ex_med      56
chargeoff_within_12_mths        56
pub_rec_bankruptcies           697
tax_liens                       39
dtype: int64

In [122]:
df.drop(["pub_rec_bankruptcies"], axis=1, inplace=True)

In [123]:
df.dropna(subset=df.columns, inplace=True)

In [124]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col, axis=1, inplace=True)

In [125]:
df.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'last_credit_pull_d'],
      dtype='object')

In [126]:
df.dtypes.value_counts()

object     11
float64    10
int64       1
dtype: int64

In [127]:
object_columns = df.select_dtypes("object")

In [128]:
object_columns.head()

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016
1,60 months,15.27%,< 1 year,RENT,Source Verified,car,bike,GA,Apr-1999,9.4%,Sep-2013
2,36 months,15.96%,10+ years,RENT,Not Verified,small_business,real estate business,IL,Nov-2001,98.5%,Jun-2016
3,36 months,13.49%,10+ years,RENT,Source Verified,other,personel,CA,Feb-1996,21%,Apr-2016
5,36 months,7.90%,3 years,RENT,Source Verified,wedding,My wedding loan I promise to pay back,AZ,Nov-2004,28.3%,Jan-2016


In [129]:
uniques = {col:len(df[col].unique()) for col in object_columns.columns}

In [130]:
uniques

{'term': 2,
 'int_rate': 371,
 'emp_length': 11,
 'home_ownership': 5,
 'verification_status': 3,
 'purpose': 14,
 'title': 18836,
 'addr_state': 50,
 'earliest_cr_line': 514,
 'revol_util': 1086,
 'last_credit_pull_d': 104}

In [131]:
df.term.head(3)

0     36 months
1     60 months
2     36 months
Name: term, dtype: object

In [132]:
df.int_rate.head(3)

0     10.65%
1     15.27%
2     15.96%
Name: int_rate, dtype: object

In [133]:
df.int_rate = df.int_rate.str.rstrip("%").astype("float64")
df.int_rate.head(3)

0    10.65
1    15.27
2    15.96
Name: int_rate, dtype: float64

In [134]:
df.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1, inplace=True)

In [135]:
df.revol_util.head(1)

0    83.7%
Name: revol_util, dtype: object

In [136]:
df.revol_util = df.revol_util.str.rstrip("%").astype(float)
df.revol_util.head(1)

0    83.7
Name: revol_util, dtype: float64

In [137]:
df.emp_length.value_counts()

10+ years    8542
< 1 year     4495
2 years      4296
3 years      4018
4 years      3349
5 years      3200
1 year       3166
6 years      2175
7 years      1712
8 years      1440
9 years      1226
Name: emp_length, dtype: int64

In [138]:
emp_map = {
    "emp_length": {
        "10+ years":10,
        "9 years":9,
        "8 years":8,
        "7 years":7,
        "6 years":6,
        "5 years":5,
        "4 years":4,
        "3 years":3,
        "2 years":2,
        "1 year":1,
        "< 1 year": 0,
        "n/a": 0
    }
}

In [139]:
df.emp_length.value_counts()

10+ years    8542
< 1 year     4495
2 years      4296
3 years      4018
4 years      3349
5 years      3200
1 year       3166
6 years      2175
7 years      1712
8 years      1440
9 years      1226
Name: emp_length, dtype: int64

In [140]:
df = df.replace(emp_map)

In [141]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]

In [142]:
dummy = pd.get_dummies(df[cat_columns])
df = pd.concat([df, dummy], axis=1)
df.drop(columns=cat_columns, axis=1, inplace=True)
df.columns

Index(['loan_amnt', 'int_rate', 'installment', 'emp_length', 'annual_inc',
       'loan_status', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'purpose_wedding', 'term_ 36 months',
       'term_ 60 months'],
      dtype='object')

In [143]:
df.dtypes[df.dtypes == "object"]

Series([], dtype: object)

## Analysis

In [144]:
from sklearn.linear_model import LogisticRegression

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37619 entries, 0 to 39734
Data columns (total 38 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            37619 non-null  float64
 1   int_rate                             37619 non-null  float64
 2   installment                          37619 non-null  float64
 3   emp_length                           37619 non-null  int64  
 4   annual_inc                           37619 non-null  float64
 5   loan_status                          37619 non-null  int64  
 6   dti                                  37619 non-null  float64
 7   delinq_2yrs                          37619 non-null  float64
 8   inq_last_6mths                       37619 non-null  float64
 9   open_acc                             37619 non-null  float64
 10  pub_rec                              37619 non-null  float64
 11  revol_bal                   

In [146]:
features = df.loc[:, df.columns!="loan_status"]

In [147]:
target = df["loan_status"]

In [148]:
lr = LogisticRegression(max_iter=100)

In [149]:
lr.fit(features, target)

/Users/deniz/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [150]:
predictions = lr.predict(features)

In [151]:
from sklearn.model_selection import cross_val_predict

In [152]:
df.shape

(37619, 38)

In [153]:
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

/Users/deniz/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/deniz/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

### Error Metric Setting

In [189]:
fp_filter = (predictions == 1) & (df["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (df["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (df["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (df["loan_status"] == 0)
tn = len(predictions[tn_filter])

accuracy = tp / (tp + fp)
print("accuracy", accuracy)

accuracy 0.8576428087604425


In [190]:
print(df[df.loan_status==1].shape[0], df[df.loan_status==0].shape[0])

32236 5383


In [159]:
lr = LogisticRegression(class_weight="balanced")

In [160]:
predictions2 = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions2)

/Users/deniz/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [161]:
penalty = {
    0: 10,
    1: 1
}
lr = LogisticRegression(class_weight=penalty)

In [162]:
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

/Users/deniz/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [188]:
fp_filter = (predictions == 1) & (df["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (df["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (df["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (df["loan_status"] == 0)
tn = len(predictions[tn_filter])

accuracy = tp / (tp + fp)
print("accuracy", accuracy)

accuracy 0.8576428087604425


In [166]:
from sklearn.ensemble import RandomForestClassifier

In [167]:
lr = RandomForestClassifier(random_state=42, class_weight="balanced")

In [168]:
features.columns

Index(['loan_amnt', 'int_rate', 'installment', 'emp_length', 'annual_inc',
       'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'home_ownership_MORTGAGE',
       'home_ownership_NONE', 'home_ownership_OTHER', 'home_ownership_OWN',
       'home_ownership_RENT', 'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'purpose_wedding', 'term_ 36 months',
       'term_ 60 months'],
      dtype='object')

In [169]:
features.columns[:13]

Index(['loan_amnt', 'int_rate', 'installment', 'emp_length', 'annual_inc',
       'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc'],
      dtype='object')

In [170]:
features.columns[13:]

Index(['home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'purpose_wedding', 'term_ 36 months',
       'term_ 60 months'],
      dtype='object')

In [171]:
features

,loan_amnt,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,27.65,0.0,1.0,3.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,1.00,0.0,5.0,3.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,8.72,0.0,2.0,2.0,0.0,...,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,20.00,0.0,1.0,10.0,0.0,...,0,0,0,1,0,0,0,0,1,0
5,5000.0,7.90,156.46,3,36000.0,11.20,0.0,3.0,9.0,0.0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39730,3000.0,7.75,93.67,9,50000.0,5.35,0.0,0.0,17.0,0.0,...,0,0,0,0,0,0,1,0,1,0
39731,3000.0,7.75,93.67,10,125000.0,2.14,0.0,0.0,15.0,0.0,...,0,0,0,0,0,0,0,0,1,0
39732,4000.0,10.91,130.79,1,18000.0,18.00,0.0,1.0,4.0,0.0,...,0,0,0,0,0,0,0,0,1,0
39733,2000.0,8.70,63.32,0,70000.0,6.07,0.0,1.0,13.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [172]:
features.iloc[:, :13] = (features.iloc[:, :13] - features.iloc[:, :13].min()) / (features.iloc[:, :13].max() - features.iloc[:, :13].min()) 

/var/folders/0p/zxhq4z1j2wv76v8ysg1rhnc00000gn/T/ipykernel_6455/263539453.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.iloc[:, :13] = (features.iloc[:, :13] - features.iloc[:, :13].min()) / (features.iloc[:, :13].max() - features.iloc[:, :13].min())


In [173]:
features

,loan_amnt,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,0.130435,0.272822,0.114137,1.0,0.003336,0.921974,0.0,0.125,0.023810,0.0,...,0,0,0,0,0,0,0,0,1,0
1,0.057971,0.513824,0.034230,0.0,0.004336,0.033344,0.0,0.625,0.023810,0.0,...,0,0,0,0,0,0,0,0,0,1
2,0.055072,0.549817,0.053230,1.0,0.001376,0.290764,0.0,0.250,0.000000,0.0,...,0,0,0,0,0,1,0,0,1,0
3,0.275362,0.420970,0.250965,1.0,0.007538,0.666889,0.0,0.125,0.190476,0.0,...,0,0,0,1,0,0,0,0,1,0
5,0.130435,0.129369,0.109166,0.3,0.005337,0.373458,0.0,0.375,0.166667,0.0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39730,0.072464,0.121544,0.060473,0.9,0.007672,0.178393,0.0,0.000,0.357143,0.0,...,0,0,0,0,0,0,1,0,1,0
39731,0.072464,0.121544,0.060473,1.0,0.020180,0.071357,0.0,0.000,0.309524,0.0,...,0,0,0,0,0,0,0,0,1,0
39732,0.101449,0.286385,0.089259,0.1,0.002335,0.600200,0.0,0.125,0.047619,0.0,...,0,0,0,0,0,0,0,0,1,0
39733,0.043478,0.171101,0.036937,0.0,0.011007,0.202401,0.0,0.125,0.261905,0.0,...,0,0,0,0,0,0,0,0,1,0


In [174]:
features.describe()

,loan_amnt,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
count,37619.000000,37619.000000,37619.000000,37619.000000,37619.000000,37619.000000,37619.000000,37619.000000,37619.000000,37619.000000,...,37619.000000,37619.000000,37619.000000,37619.000000,37619.000000,37619.000000,37619.000000,37619.000000,37619.000000,37619.000000
mean,0.308929,0.342083,0.240057,0.495005,0.010913,0.443471,0.013366,0.109034,0.173982,0.013437,...,0.055371,0.017411,0.014620,0.098487,0.002499,0.045562,0.009197,0.024323,0.749036,0.250964
std,0.213594,0.192464,0.161997,0.355782,0.010778,0.222074,0.044840,0.133897,0.104641,0.058663,...,0.228706,0.130800,0.120029,0.297977,0.049926,0.208536,0.095463,0.154051,0.433573,0.433573
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.144928,0.188837,0.117906,0.200000,0.006171,0.272758,0.000000,0.000000,0.095238,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.275362,0.334377,0.205723,0.400000,0.009340,0.446482,0.000000,0.125000,0.166667,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.420290,0.471570,0.321528,0.900000,0.013175,0.618873,0.000000,0.125000,0.238095,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [175]:
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

In [184]:
fp_filter = (predictions == 1) & (df["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (df["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (df["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (df["loan_status"] == 0)
tn = len(predictions[tn_filter])

accuracy = tp / (tp + fp)
print("accuracy", accuracy)

accuracy 0.8576428087604425


In [183]:
for k in [3, 5, 10]:
    print(f"For k: {k}")
    predictions = cross_val_predict(lr, features, target, cv=k)
    predictions = pd.Series(predictions)
    fp_filter = (predictions == 1) & (df["loan_status"] == 0)
    fp = len(predictions[fp_filter])

    # True positives.
    tp_filter = (predictions == 1) & (df["loan_status"] == 1)
    tp = len(predictions[tp_filter])

    # False negatives.
    fn_filter = (predictions == 0) & (df["loan_status"] == 1)
    fn = len(predictions[fn_filter])

    # True negatives
    tn_filter = (predictions == 0) & (df["loan_status"] == 0)
    tn = len(predictions[tn_filter])

    accuracy = tp / (tp + fp)
    print("accuracy", accuracy)

For k: 3
accuracy 0.857534169208178
For k: 5
accuracy 0.8576225275810502
For k: 10
accuracy 0.8576428087604425


In [182]:
fold_count = 10